In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

In [25]:
import xarray as xr

filename = "OBS6_ERA5_reanaly_1_day_pr_2010-2012.nc"  # Update the path to the correct location of the file
# Read the NetCDF file
ds = xr.open_dataset(filename)
# Convert to a pandas DataFrame
df = ds.to_dataframe()
# Reset the index if needed
df.reset_index(inplace=True)
df = df[df['bnds'] == 0]
df.index = pd.to_datetime(df['time'])
df.index = df.index.date
df.drop(columns=['time', 'bnds'], inplace=True)
prec  = df['pr']  # Precipitation data
prec  = prec*86400  # Convert from m/s to mm/day)
prec[prec < 0] = 0  # Set negative values to zero


filename = "Derived_Makkink_evspsblpot.nc"  # Update the path to the correct location of the file
# Read the NetCDF file
dsEP = xr.open_dataset(filename)
# Convert to a pandas DataFrame
dfEP = dsEP.to_dataframe()
# Reset the index if needed
dfEP.reset_index(inplace=True)
EP = dfEP['evspsblpot']  # Evapotranspiration data
EP.index = pd.to_datetime(dfEP['time'])
EP.index = EP.index.date
EP = EP*86400  # Convert from mm/s to mm/day)
EP[EP < 0] = 0  # Set negative values to zero

filename = "6227510_Q_Day.Cmd.txt"
A = 21497*10**6 #m^2
dfQ = pd.read_csv(filename, delimiter=";", skiprows=37, header=None, encoding='latin1')  # Specify encoding
dfQ = dfQ[[0, 2]]  # Select only the first and third columns
dfQ.columns = ['Date', 'Q']  # Rename columns for clarity
dfQ['Date'] = pd.to_datetime(dfQ['Date'], format='%Y-%m-%d')  # Convert 'Date' column to datetime
dfQ.set_index('Date', inplace=True)  # Set 'Date' as the index
dfQ['Q'] = (dfQ['Q']/A)*(10**3)*86400 #mm/day
dfQ = dfQ[(dfQ.index >= '2010-01-01') & (dfQ.index <= '2012-12-31')]  # Filter the DataFrame for the desired date range
dfQ

,Q
Date,
2010-01-01,0.061734
2010-01-02,0.061252
2010-01-03,0.059644
2010-01-04,0.066638
2010-01-05,0.066397
...,...
2012-12-27,0.068165
2012-12-28,0.066517
2012-12-29,0.065834


In [26]:
n = len(prec)
t = np.arange(n)
Si1 = np.zeros(n)
Si2 = np.zeros(n)
Si_max = 2.5 #?????????????????????????????????????????????????????????????????''
Pe_1 = np.zeros(n)
Pe_2 = np.zeros(n)

for t in np.arange(1,n,1):

    Si1[t] = Si1[t-1] + prec.iloc[t]  # Use .iloc to get the scalar value
    if Si1[t] > Si_max:
        Pe_1[t] = Si1[t] - Si_max
    else:
        Pe_1[t] = 0
    Si1[t] = Si1[t] - Pe_1[t]
    deltS = Si1[t] - EP.iloc[t]  # Use .iloc to get the scalar value
    Si1[t] = np.maximum(0, deltS)

Pe_1_mean = Pe_1.mean()


print(Pe_1_mean/prec.mean())


0.6123322898736734


In [27]:
print(len(prec.index))
print(len(EP.index))
print(len(dfQ.index))
print(len(Si1))
print(len(Pe_1))


1096
1096
1096
1096
1096


In [28]:
data = pd.DataFrame({'P': prec, 'EP': EP, 'Q': dfQ['Q'].values, 'S': Si1, 'Pe': Pe_1}, index=prec.index)
data.head(15)

,P,EP,Q,S,Pe
2010-01-01,1.570316,0.999857,0.061734,0.000000,0.000000
2010-01-02,0.038551,0.846168,0.061252,0.000000,0.000000
2010-01-03,2.237453,0.999148,0.059644,1.238304,0.000000
2010-01-04,11.731629,0.480451,0.066638,2.019549,10.469933
2010-01-05,6.421555,0.708420,0.066397,1.791580,5.941104
2010-01-06,1.186198,0.972899,0.065151,1.527101,0.477778
2010-01-07,22.014578,0.136654,0.079258,2.363346,21.041680
2010-01-08,4.502331,0.781251,0.128613,1.718749,4.365676
2010-01-09,0.083060,0.915587,0.098550,0.886223,0.000000
2010-01-10,4.330080,0.875250,0.082112,1.624750,2.716302


In [29]:
P = data['P'].mean()
Pe = data['Pe'].mean()

print(Pe/prec.mean())

0.6123322898736734


In [35]:
#PROBLEM 2
P_mean = data['P'].mean()
EP_mean = data['EP'].mean()
Q_mean = data['Q'].mean()

ET_mean = P_mean - Q_mean #transpiration

#print (P_mean, EP_mean, Q_mean, ET_mean)

data.loc[:,'ET'] = (data['EP']/EP_mean)*ET_mean

data.loc[:,'P-ET'] = data['P'] - data['ET']


for i in range(1, len(data)):
    if data['P-ET'].iloc[:i].cumsum().iloc[-1] > 0:
        data.loc[data.index[i], 'SD'] = 0
    else:
        data.loc[data.index[i], 'SD'] = data['P-ET'].iloc[:i].cumsum().iloc[-1]


SR = data['SD'].min()
        
print(-SR)

93.78133351803284
